# 1. Basic Wolfram Language Integration with Python

Continuation of _Wolfram Language for Python Users_. This notebook contains examples of:

- generating and executing Wolfram Language code
- accessing the Wolfram Knowledgebase
- communication between languages

## Generating Wolfram Language code

All named Wolfram expressions (symbols, functions) can be represented in Python using the **wl.** header:

In [ ]:
from wolframclient.language import wl, wlexpr
x = wl.Select(wl.Range(10), wl.PrimeQ)
y = wl.Reverse(wl.Range(10))
z = wl.Now
print(x)
print(y)
print(z)

**wl.expr** can be used to represent all unnamed Wolfram expressions:

In [ ]:
wl.Map(wlexpr('#^2&'), [1,2,3])

The syntax of WL functions must be modified to work with Python. For example:

Function[arg1, arg2, Option->OptionValue] becomes wl.Function(arg1, arg2, Option=OptionValue).

## Evaluating Wolfram Language code

Wolfram expressions generated inside Python will not evaluate unless called explicitly. One way to achieve this is with local kernel evaluation, which utilises the Wolfram kernel licences contained within a Wolfram product installation such as Mathematica. This is achieved with **session.evaluate** imported from the **WolframLanguageSession** library:

In [ ]:
from wolframclient.evaluation import WolframLanguageSession
session = WolframLanguageSession() # May need to provide a file path. See https://wolfr.am/177vH2umJ

session.evaluate(x)

In [ ]:
session.evaluate(y)

In [ ]:
session.evaluate(z)

**Data Conversion**

As shown above, some simple data types (lists, integers) are automatically converted into Python formats. Others, such as DateObject and Graphic, will remain as WL expression types and will need to be serialised (see section 2), manually converted or exported/rendered:

In [ ]:
import os
data = [.3, 2, 4, 5, 5.5]
graphic = wl.BarChart(data, ChartLabels=["a", "b", "c", "d", "e"])
path = os.path.join(os.getcwd(), 'AdditionalFiles')
png_export = wl.Export(os.path.join(path, 'barchart1.png'), graphic, "PNG")
session.evaluate(png_export)

Some data types can be converted between Python and WL once a relevant library is loaded. For example, PIL or iPython images can be used as image inputs for WL functions:

In [ ]:
from PIL import Image
barchartImage = Image.open(os.path.join(path, 'barchart1.png'))
barchartImage

In [ ]:
png_export2 = wl.Export(os.path.join(path, 'barchart2.png'), wl.GradientFilter(barchartImage, 3), "PNG")
session.evaluate(png_export2)
Image.open(os.path.join(path, 'barchart2.png'))

## Defining Functions

In addition to symbol definition, Python functions can be defined using WL functions. This can be achieved either by defining the function using Python syntax and WL headers, or by using **session function** to read a pre-defined function within the Wolfram kernel.
 
**Example:** Define a function to return the population density as a fraction of people to country area in people/miles^2.

**Method 1: using wl headers**

In [9]:
def population_density(country):
    return(session.evaluate(
        wl.QuantityMagnitude(
            wl.Divide(
                wl.EntityValue(wl.Entity("Country", country), "Population"),
                wl.EntityValue(wl.Entity("Country", country), "Area")
            )
        )
  ))

In [ ]:
population_density("France")

**Method 2: using WL string expression and session function**

First evaluate a WL function definition:

In [11]:
session.evaluate(wlexpr('''
    populationDensity[country_String] := QuantityMagnitude[
        Divide[
            EntityValue[Entity["Country", country], "Population"],
            EntityValue[Entity["Country", country], "Area"]
        ]
    ];
'''))

Then call this definition using session.function:

In [12]:
population_density2 = session.function(wlexpr('populationDensity'))

In [ ]:
population_density2("France")

Alternatively, using the Global environment:

In [ ]:
from wolframclient.language import Global
population_density3 = session.function(Global.populationDensity)
population_density3("France")

## Sessions

**Note:** Sessions are equivalent to Wolfram kernels and if not properly terminated will form orphaned kernels. Aim to maintain only a single session within Python and terminate it at the end of every session.

To terminate a session, use session.terminate():

In [ ]:
session.terminate()

Alternatively, a session can be handled locally with a With block:

In [ ]:
with WolframLanguageSession() as wl_blocksession: # May need to provide a file path. See https://wolfr.am/177vH2umJ
    print(wl_blocksession.evaluate(wl.StringReverse('abc')))